In [25]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

In [16]:
# Глобальные константы, прописываем в отдельные преременные для удобства

RANDOM_SEED = 42
PATH_TO_RAW_DATA = os.path.join('data', 'raw', 'geo-reviews-dataset-2023.csv')

In [19]:
# Загружаем датасэт

raw_data_set = pd.read_csv(PATH_TO_RAW_DATA)
print(f'Объем сырого датасэта: {len(dataset)}')
raw_data_set.head(5)

Объем сырого датасэта: 500000


,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [ ]:
labels = raw_data_set['rating'].unique() # возвращает объект типа np.ndarray

In [23]:
# Бьем на train и test, defaul shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(raw_data_set['text'], raw_data_set['rating'], train_size=0.7, random_state=RANDOM_SEED, stratify=raw_data_set['rating'])

In [ ]:
# Токенизируем тексты

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

print(train_tokens.keys())

print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self,
                 text: np.array,
                 lable: pd.DataFrame):
        self.profile = FloatTensor(profile)
        self.group = group

    def __len__(self):
        return int(self.profile.size(dim = 0))

    def __getitem__(self, idx: int) -> tuple[Union[FloatTensor, str], Union[FloatTensor, str]]:
        return self.profile[idx, :], self.group[idx], self.name[idx]

    def subset(self, indices: list) -> tuple[np.array, pd.Series, pd.Series]:
        profiles = self.profile[indices, :]
        groups = self.group.iloc[indices].reset_index(drop=True)
        names = self.name.iloc[indices].reset_index(drop=True)
        return profiles, groups, names

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')